In [268]:
# Libraries Import

In [269]:
import pandas as pd
import numpy as np
from scipy.io import arff
import ipywidgets as widgets
from IPython.display import Markdown, SVG
from datetime import datetime
import random
import urllib.request, json 
import gspread

In [270]:
# Dataset Load
data = arff.loadarff("./datasets/WINE.txt.arff")
dataset = pd.DataFrame(data[0])

In [271]:
def labelConversion(label):
    if label == b'1':
        return '1'
    elif label == b'2':
        return '2'
    else:
        return '3'

dataset['Class'] = dataset['Class'].apply(lambda x: labelConversion(x))

In [272]:
# "Ingredients" for the preparation of the survey 

In [273]:
questions_numbers = [n for n in range(5)]

In [274]:
def warm_upAnswers(current_web_page):
    if current_web_page.__name__ == 'warmUpWebPage':
        if example_selection_1.value != None and example_selection_2.value != None:
            return True
        else:
            return False

In [275]:
next_button = widgets.Button(
    description='Next'
)

submit_button = widgets.Button(
    description='Submit'
)

output = widgets.Output()

In [276]:
# Preparation of the Questions section of the survey

questions_selection = []
for x in range(5):
    questions_selection.append(widgets.RadioButtons(
        options=[('Class 1', '1'), ('Class 2', '2'), ('Class 3', '3')],
        value=None,
        disabled=False
    ))

questions_samples = [dataset.iloc[0],dataset.iloc[58],dataset.iloc[164],dataset.iloc[177],dataset.iloc[156]]
correct_classifications = []

random.shuffle(questions_samples)

for n,sample in zip(range(5),questions_samples.copy()):
    correct_classifications.append(questions_samples[n][-1])
    questions_samples[n] = questions_samples[n][:-1]

In [277]:
gender_selection = widgets.RadioButtons(
    options=[('Male', '1'), ('Female', '2'), ('Other', '3')],
    value=None,
    disabled=False
)

age_selection = widgets.RadioButtons(
    options=[('18-20', '1'), ('21-29', '2'), ('30-39', '3'),('40-49', '4'),('50-59', '5'),('60 or older', '6')],
    value=None,
    disabled=False
)

education_selection = widgets.RadioButtons(
    options=[('Less than high school degree', '1'), ('High school degree or equivalent', '2'), ('Bachelor degree', '3'),('Graduate degree', '4')],
    value=None,
    disabled=False
)

english_level_selection = widgets.RadioButtons(
    options=[('A1', '1'), ('A2', '2'), ('B1', '3'), ('B2', '4'), ('C1', '5'), ('C2', '6')],
    value=None,
    disabled=False
)

In [278]:
participant_suggestion_text_area = (widgets.Textarea(
        value='',
        disabled=False
    ))

In [279]:
def welcomeWebPage():
    output.clear_output()
    with output:
        display(Markdown("# Survey of the interpretability of decision trees"))
        display(Markdown('''This questionnaire aims to evaluate the interpretability of decision trees. We want to understand what makes a decision tree more interpretable than others. 
        This survey uses an activity-based rating in which we ask you to solve a problem, and each question comes with a few rating questions related to the activity.
        \nThis study is done for research purposes and carried out by researchers from the University of Bolzano. We collect information that agrees with the European Union's General Data Protection Regulation (GDPR). The data collected is for research purposes only and based on non-personal or anonymous data provided during your voluntary participation.
        \nThe survey takes 10 minutes to complete.
        \nIn case of questions or doubts, send an email to Marco.Zenere@stud-inf.unibz.it.'''))
        display(Markdown('''By clicking the next button, you partecipate to the questionnaire and confirm that:
        \n- You have reached the age of majority
        \n- You acknowledge that your participation is completely voluntary
        \n- You acknowledge that your anonymous responses may be used for research purposes in accordance with General Data Protection Regulation'''))
        display(next_button)
    display(output)

In [280]:
def introductionWebPage():
    
    output.clear_output()
    with output:
        display(Markdown('## Introduction'))
        display(Markdown('''The data used in this questionnaire is part of the wine dataset provided by UCI. These data are the results of a chemical analysis of wines grown in the same region in Italy but derived from three different cultivars. The analysis determined the quantities of 13 constituents found in each of the three types of wines.
                            \n- Alcohol
                            \n- Malic acid
                            \n- Ash
                            \n- Alcalinity of ash
                            \n- Magnesium
                            \n- Total phenols
                            \n- Flavanoids
                            \n- Nonflavanoid phenols
                            \n- Proanthocyanins
                            \n- Color intensity
                            \n- Hue
                            \n- OD280/OD315 of diluted wines
                            \n- Proline

                            \nThis questionnaire considers decision tree representation in the form depicted below.'''))
        
        display(SVG(filename='./Images/Decision_Tree_3_layers.svg'))
        
        display(Markdown('### How to interpret the decision tree?'))
        
        display(Markdown('''A decision tree is a tree-like model that checks one attribute/feature at a time of a given instance to classify it in one of the classes of the domain of interest. 
                         A decision tree is made up of several levels and nodes. Each level is composed of one or more nodes, and the nodes correspond to the features of the domain of interest. 
                         Typically, a decision tree considers part of the available features, which correspond to those most relevant to correctly classifying an instance of the domain.  
                         Each node evaluates whether the value of the considered attribute is above a specific threshold. If the attribute value exceeds the threshold, the decision tree will test the attribute on the right path. Otherwise, it will test the one on the left path. 
                         In the decision tree representation of the following survey, the threshold of each node is represented by the black triangle in the histogram, and each node is a histogram. To classify an instance of the domain of interest, the decision tree must reach one of the leaf nodes starting from the root node. A leaf node could be impure, so there could be the case where the leaf node contains multiple classes. 
                         The decision tree representation in the following survey highlights the composition of each leaf node and indicates the dominant class.'''))
        display(Markdown("Let's take a look at an example."))
        display(next_button)

In [281]:
def exampleWebPage():
    output.clear_output()
    with output:
        display(Markdown('## Example'))
        display(Markdown('Considering we have a wine instance with the following features:'))
        # Example 1
        example = dataset.iloc[1][:-1].to_frame()
        example.rename(columns = {example.columns[0]:''}, inplace = True)
        display(example.transpose())
        # SVG Image
        display(SVG(filename='./Images/Decision_Tree_3_layers.svg'))
        # Explanation
        display(Markdown('''Considering the above decision tree representation and starting from the top level of the decision tree, the first feature to consider is the Proline. 
        The value of the example is above the threshold of 760.0, so we need to take the right path of the decision tree and continue to go through the tree. 
        The next feature we need to consider is Flavanoids, and the example has a value above the threshold of 2.17 and as before we need to take the right path of the decision tree and continue to go through the tree.
        The last feature to consider before the classification of the example is Magnesium. Our example has a value under the threshold of 135.50, and by taking the left path of the decision tree, we can conclude that the wine is of Class 1.'''))
        display(SVG(filename='./Images/Decision_Tree_3_layers_answer.svg'))
        display(Markdown("Before starting the survey, let's do a quick warm-up to understand whether or not the concept is clear."))
        display(next_button)

In [282]:
def warmUpWebPage():
    output.clear_output()
    with output:
        display(Markdown('## Warm-Up'))
        display(Markdown('Considering we have a wine with the following features:'))
        # Example 96
        example = dataset.iloc[96][:-1].to_frame()
        example.rename(columns = {example.columns[0]:''}, inplace = True)
        display(example.transpose())
        # SVG Image
        display(Markdown("Let's consider the decision tree representation below."))
        display(SVG(filename='./Images/Decision_Tree_3_layers.svg'))
        # Warm-Up Questions
        display(Markdown(' ### Which class correspond the wine with the following features ?'))
        example_selection_1 = widgets.RadioButtons(
                                options=[('Class 1', '1'), ('Class 2', '2'), ('Class 3', '3')],
                                value=None,
                                disabled=False
                                )
        display(example_selection_1)
        display(Markdown('### Which of the following features/attributes did you consider for the classification?'))
        example_selection_2 = widgets.RadioButtons(
                                options=[('Proline, OD280/OD315, and Flavanoids', '1'), ('Proline, OD280/OD315, and Hue', '2'), ('Proline, Flavanoids, and Magnesium', '3'), ('Proline, Flavanoids, and Hue', '4')],
                                value=None,
                                disabled=False
                                )
        display(example_selection_2)
        display(Markdown('The warm-up is completed. Click the next button to read the instructions of the survey.'))
        display(next_button)

In [283]:
def questionnaireInstructionWebPage():
    output.clear_output()
    with output:
        display(Markdown('## Survey Instruction'))
        display(Markdown('''The questionnaire is composed of five questions, and each of them is composed of the following elements:
                            \n- A wine instance features
                            \n- Two decision tree representations of the same domain of interest
                            \n- A set of mandatory questions: three questions for evaluating the decision tree representations, one for specifying the classification of the given wine instance
                            \n- An optional section for leaving additional comments

                            \nAfter completing the five questions, there will be four questions to understand who you are, to be able to do a demographic analysis of the participants.

                            \nYou can start the survey.'''))
        display(next_button)

In [284]:
def questionWebPage():
    output.clear_output()
    with output:
        question_number = questions_numbers.pop(0)
        display(Markdown('## Question %s'%(str(question_number + 1))))
        # Wine instance extraction from preselected sample
        display(Markdown('''Considering a wine instance with the following features:'''))
        sample_DataFrame = questions_samples.pop(0).to_frame()
        sample_DataFrame.rename(columns = {sample_DataFrame.columns[0]:''}, inplace = True)
        display(sample_DataFrame.transpose())
        display(Markdown("Let's consider the following decision tree representation:"))
        # Random selection of a decision tree representation
        image_selector = random.randint(0,100)
        if image_selector % 2 == 0 :
            display(SVG(filename='./Images/Decision_Tree_3_layers.svg'))
        else:
            display(SVG(filename='./Images/Decision_Tree_5_layers.svg'))
        # Radio Button/Question
        display(Markdown("### Which class correspond the wine with the following features?"))
        display(questions_selection[question_number])
        display(next_button)

In [285]:
def participantCommentWebPage():
    output.clear_output()
    with output:
        display(Markdown('## Additional Comment'))
        display(Markdown('If you have a comment or suggestion regarding the questionnaire you want to tell us, please fill the text box below.'))
        display(participant_suggestion_text_area)
        display(next_button)

In [286]:
def participatInfoWebPage():
    output.clear_output()
    with output:
        display(Markdown('## Information about the survey participant'))
        display(Markdown('We would like to collect some information about you to do a demographic analysis of the participants in our survey.'))
        display(Markdown('### Gender'))
        display(gender_selection)
        display(Markdown('### Age'))
        display(age_selection)
        display(Markdown('### Education'))
        display(education_selection)
        display(Markdown('### English Level'))
        display(english_level_selection)
        display(submit_button)

In [287]:
def endquestionnaireWebPage():
    output.clear_output()
    with output:
        display(Markdown('## The questionnaire is concluded'))
        display(Markdown('The Survey was submitted correctly. You can close the browser tab. Thanks for the partecipation'))

In [288]:
web_pages_order = [introductionWebPage, exampleWebPage, warmUpWebPage, questionnaireInstructionWebPage, questionWebPage, questionWebPage, questionWebPage, questionWebPage, questionWebPage, participantCommentWebPage, participatInfoWebPage, endquestionnaireWebPage]

In [289]:
web_pages = ['introductionWebPage', 'exampleWebPage']

In [290]:

def next_clicked(b):
    web_page = web_pages_order.pop(0)
    test = web_pages.pop(0)
    with output:
        # TO FIX
        if test == 'exampleWebPage':
            print(test)
            web_page()

def submit_clicked(b):
    web_page = web_pages_order.pop(0)
    web_page()
    
next_button.on_click(next_clicked)
submit_button.on_click(submit_clicked)

In [291]:
welcomeWebPage()

Output()